In [ ]:
%load_ext lab_black

In [1]:
import pandas as pd

from sadie.anarci import Anarci
from sadie.anarci.numbering import scheme_numbering
import gzip

import Bio
from random import sample

In [21]:
anarci_api = Anarci(scheme="kabat", region_assign="kabat", allowed_species=["human"],threshold=100)
result = anarci_api.run_single(
    "VRC26.27_KT371104_Homo_sapiens_anti-HIV-1_immunoglobulin",
    "QVQLVESGGGVVQPGTSLRLSCAASQFRFDGYGMHWVRQAPGKGLEWVASISHDGIKKYHAEKVWGRFTISRDNSKNTLYLQMNSLRPEDTALYYCAKDLREDECEEWWSDYYDFGKQLPCAKSRGGLVGIADNWGQGTMVTVS",
)

In [22]:
result['cdr3_aa_gaps']

0    DLREDECEEWWSDYYDFGKQLPCAKSRGGLVGIADN
Name: cdr3_aa_gaps, dtype: object

In [27]:
at['100Z'],at['101']

(0    G
 Name: 100Z, dtype: object,
 0    D
 Name: 101, dtype: object)

In [23]:
at = result.get_alignment_table()

In [24]:
list(at.columns)

['Id',
 'chain_type',
 'scheme',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '52A',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '82A',
 '82B',
 '82C',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '100A',
 '100B',
 '100C',
 '100D',
 '100E',
 '100F',
 '100G',
 '100H',
 '100I',
 '100J',
 '100K',
 '100L',
 '100M',
 '100N',
 '100O',
 '100P',
 '100Q',
 '100R',
 '100S',
 '100T',
 '100U',
 '100V',
 '100W',
 '100X',
 '100Y',
 '100Z',
 '100a',
